<a href="https://colab.research.google.com/github/kyochanpy/Kaggle_Indoor_Location_Navigation/blob/main/create_dataset/sensor_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import shutil
from scipy import signal


In [2]:
# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

#acc_50

In [ ]:
#acc_x_train
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv')
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    acc_df = pd.read_csv(f'/content/drive/MyDrive/accelerometer/{building}_accelerometer_train.csv')
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        if len(timestamp_list) == 0:
            break
        else:
            timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            else:
                timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
                timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            else:
                acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(acc_path_timestamp_df['x'].values) == 0:
                break
            else:
                acc_path_timestamp_list = acc_path_timestamp_df['x'].values
                acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                    np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
                acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
                acc_path_timestamp_df_.columns = [f'acc_x_{str(i)}' for i in range(100)]
                acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
                acc_path_timestamp_df_['path'] = path
                dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/acc_100/x')
    print(building)

5a0546857ecc773753327266
5c3c44b80379370013e0fd2b
5d27075f03f801723c2e360f
5d27096c03f801723c31e5e0
5d27097f03f801723c320d97
5d27099f03f801723c32511d
5d2709a003f801723c3251bf
5d2709b303f801723c327472
5d2709bb03f801723c32852c
5d2709c303f801723c3299ee
5d2709d403f801723c32bd39


In [ ]:
#acc_x_test
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/accelerometer/{building}_accelerometer_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['x'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['x'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'acc_x_{str(i)}' for i in range(100)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/acc_100/x')

In [ ]:
#acc_y_train
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv')
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    acc_df = pd.read_csv(f'/content/drive/MyDrive/accelerometer/{building}_accelerometer_train.csv')
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        if len(timestamp_list) == 0:
            break
        else:
            timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            else:
                timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
                timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            else:
                acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(acc_path_timestamp_df['y'].values) == 0:
                break
            else:
                acc_path_timestamp_list = acc_path_timestamp_df['y'].values
                acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                    np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
                acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
                acc_path_timestamp_df_.columns = [f'acc_y_{str(i)}' for i in range(100)]
                acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
                acc_path_timestamp_df_['path'] = path
                dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/acc_100/y')
    print(building)

In [ ]:
#acc_y_test
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/accelerometer/{building}_accelerometer_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['y'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['y'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'acc_y_{str(i)}' for i in range(100)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/acc_100/y')

In [ ]:
#acc_z_train
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv')
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    acc_df = pd.read_csv(f'/content/drive/MyDrive/accelerometer/{building}_accelerometer_train.csv')
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        if len(timestamp_list) == 0:
            break
        else:
            timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            else:
                timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
                timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            else:
                acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            else:
                acc_path_timestamp_list = acc_path_timestamp_df['z'].values
                acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                    np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
                acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
                acc_path_timestamp_df_.columns = [f'acc_z_{str(i)}' for i in range(100)]
                acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
                acc_path_timestamp_df_['path'] = path
                dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/acc_100/z')
    print(building)

5a0546857ecc773753327266
5c3c44b80379370013e0fd2b
5d27075f03f801723c2e360f
5d27096c03f801723c31e5e0
5d27097f03f801723c320d97
5d27099f03f801723c32511d
5d2709a003f801723c3251bf
5d2709b303f801723c327472
5d2709bb03f801723c32852c
5d2709c303f801723c3299ee
5d2709d403f801723c32bd39
5d2709e003f801723c32d896
5da138274db8ce0c98bbd3d2
5da1382d4db8ce0c98bbe92e
5da138314db8ce0c98bbf3a0
5da138364db8ce0c98bc00f1
5da1383b4db8ce0c98bc11ab
5da138754db8ce0c98bca82f
5da138764db8ce0c98bcaa46
5da1389e4db8ce0c98bd0547
5da138b74db8ce0c98bd4774
5da958dd46f8266d0737457b
5dbc1d84c1eb61796cf7c010
5dc8cea7659e181adb076a3f


In [ ]:
#acc_z_test
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/accelerometer/{building}_accelerometer_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'acc_z_{str(i)}' for i in range(100)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/acc_100/z')

#gyro_50

In [ ]:
#gyro_x_train
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv')
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    gyro_df = pd.read_csv(f'/content/drive/MyDrive/gyroscope/{building}_gyroscope_train.csv')
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        gyro_path_df = gyro_df[gyro_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        if len(timestamp_list) == 0:
            break
        else:
            timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            else:
                timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
                timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            else:
                gyro_path_timestamp_df = gyro_path_df[(gyro_path_df['timestamp'] >= timestamp_05_list[n]) & (gyro_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(gyro_path_timestamp_df['x'].values) == 0:
                break
            else:
                gyro_path_timestamp_list = gyro_path_timestamp_df['x'].values
                gyro_path_timestamp_list_ = np.interp(np.arange(0, len(gyro_path_timestamp_list), len(gyro_path_timestamp_list) * 0.0101),
                                                    np.arange(0, len(gyro_path_timestamp_list)), gyro_path_timestamp_list)
                gyro_path_timestamp_df_ = pd.DataFrame(gyro_path_timestamp_list_).T
                gyro_path_timestamp_df_.columns = [f'gyro_x_{str(i)}' for i in range(100)]
                gyro_path_timestamp_df_['timestamp'] = timestamp_list[n]
                gyro_path_timestamp_df_['path'] = path
                dfs_.append(gyro_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/gyro_100/x')
    print(building)

In [ ]:
#gyro_x_test
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/gyroscope/{building}_gyroscope_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['x'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['x'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'gyro_x_{str(i)}' for i in range(100)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/gyro_100/x')

In [ ]:
#gyro_y_train
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv')
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    gyro_df = pd.read_csv(f'/content/drive/MyDrive/gyroscope/{building}_gyroscope_train.csv')
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        gyro_path_df = gyro_df[gyro_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        if len(timestamp_list) == 0:
            break
        else:
            timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            else:
                timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
                timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            else:
                gyro_path_timestamp_df = gyro_path_df[(gyro_path_df['timestamp'] >= timestamp_05_list[n]) & (gyro_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(gyro_path_timestamp_df['y'].values) == 0:
                break
            else:
                gyro_path_timestamp_list = gyro_path_timestamp_df['y'].values
                gyro_path_timestamp_list_ = np.interp(np.arange(0, len(gyro_path_timestamp_list), len(gyro_path_timestamp_list) * 0.0101),
                                                    np.arange(0, len(gyro_path_timestamp_list)), gyro_path_timestamp_list)
                gyro_path_timestamp_df_ = pd.DataFrame(gyro_path_timestamp_list_).T
                gyro_path_timestamp_df_.columns = [f'gyro_y_{str(i)}' for i in range(100)]
                gyro_path_timestamp_df_['timestamp'] = timestamp_list[n]
                gyro_path_timestamp_df_['path'] = path
                dfs_.append(gyro_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/gyro_100/y')
    print(building)

In [ ]:
#gyro_y_test
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/gyroscope/{building}_gyroscope_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['y'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['y'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'gyro_y_{str(i)}' for i in range(100)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/gyro_100/y')

In [ ]:
#gyro_z_train
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv')
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    gyro_df = pd.read_csv(f'/content/drive/MyDrive/gyroscope/{building}_gyroscope_train.csv')
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        gyro_path_df = gyro_df[gyro_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        if len(timestamp_list) == 0:
            break
        else:
            timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            else:
                timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
                timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            else:
                gyro_path_timestamp_df = gyro_path_df[(gyro_path_df['timestamp'] >= timestamp_05_list[n]) & (gyro_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(gyro_path_timestamp_df['z'].values) == 0:
                break
            else:
                gyro_path_timestamp_list = gyro_path_timestamp_df['z'].values
                gyro_path_timestamp_list_ = np.interp(np.arange(0, len(gyro_path_timestamp_list), len(gyro_path_timestamp_list) * 0.0101),
                                                    np.arange(0, len(gyro_path_timestamp_list)), gyro_path_timestamp_list)
                gyro_path_timestamp_df_ = pd.DataFrame(gyro_path_timestamp_list_).T
                gyro_path_timestamp_df_.columns = [f'gyro_z_{str(i)}' for i in range(100)]
                gyro_path_timestamp_df_['timestamp'] = timestamp_list[n]
                gyro_path_timestamp_df_['path'] = path
                dfs_.append(gyro_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/gyro_100/z')
    print(building)

In [ ]:
#gyro_z_test
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/gyroscope/{building}_gyroscope_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'gyro_z_{str(i)}' for i in range(100)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/gyro_100/z')

#mag_50

In [ ]:
#mag_x_train
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv')
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    mag_df = pd.read_csv(f'/content/drive/MyDrive/magnetic_field/{building}_magnetic_field_train.csv')
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        mag_path_df = mag_df[mag_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        if len(timestamp_list) == 0:
            break
        else:
            timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            else:
                timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
                timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            else:
                mag_path_timestamp_df = mag_path_df[(mag_path_df['timestamp'] >= timestamp_05_list[n]) & (mag_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(mag_path_timestamp_df['x'].values) == 0:
                break
            else:
                mag_path_timestamp_list = mag_path_timestamp_df['x'].values
                mag_path_timestamp_list_ = np.interp(np.arange(0, len(mag_path_timestamp_list), len(mag_path_timestamp_list) * 0.0101),
                                                    np.arange(0, len(mag_path_timestamp_list)), mag_path_timestamp_list)
                mag_path_timestamp_df_ = pd.DataFrame(mag_path_timestamp_list_).T
                mag_path_timestamp_df_.columns = [f'mag_x_{str(i)}' for i in range(100)]
                mag_path_timestamp_df_['timestamp'] = timestamp_list[n]
                mag_path_timestamp_df_['path'] = path
                dfs_.append(mag_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/mag_100/x')
    print(building)

In [ ]:
#mag_x_test
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/magnetic_field/{building}_magnetic_field_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['x'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['x'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'mag_x_{str(i)}' for i in range(100)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/mag_100/x')

In [ ]:
#mag_y_train
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv')
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    mag_df = pd.read_csv(f'/content/drive/MyDrive/magnetic_field/{building}_magnetic_field_train.csv')
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        mag_path_df = mag_df[mag_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        if len(timestamp_list) == 0:
            break
        else:
            timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            else:
                timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
                timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            else:
                mag_path_timestamp_df = mag_path_df[(mag_path_df['timestamp'] >= timestamp_05_list[n]) & (mag_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(mag_path_timestamp_df['y'].values) == 0:
                break
            else:
                mag_path_timestamp_list = mag_path_timestamp_df['y'].values
                mag_path_timestamp_list_ = np.interp(np.arange(0, len(mag_path_timestamp_list), len(mag_path_timestamp_list) * 0.0101),
                                                    np.arange(0, len(mag_path_timestamp_list)), mag_path_timestamp_list)
                mag_path_timestamp_df_ = pd.DataFrame(mag_path_timestamp_list_).T
                mag_path_timestamp_df_.columns = [f'mag_y_{str(i)}' for i in range(100)]
                mag_path_timestamp_df_['timestamp'] = timestamp_list[n]
                mag_path_timestamp_df_['path'] = path
                dfs_.append(mag_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/mag_100/y')
    print(building)

In [ ]:
#mag_y_test
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/magnetic_field/{building}_magnetic_field_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['y'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['y'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'mag_y_{str(i)}' for i in range(100)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/mag_100/y')

In [ ]:
#mag_z_train
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv')
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    mag_df = pd.read_csv(f'/content/drive/MyDrive/magnetic_field/{building}_magnetic_field_train.csv')
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        mag_path_df = mag_df[mag_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        if len(timestamp_list) == 0:
            break
        else:
            timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            else:
                timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
                timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            else:
                mag_path_timestamp_df = mag_path_df[(mag_path_df['timestamp'] >= timestamp_05_list[n]) & (mag_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(mag_path_timestamp_df['z'].values) == 0:
                break
            else:
                mag_path_timestamp_list = mag_path_timestamp_df['z'].values
                mag_path_timestamp_list_ = np.interp(np.arange(0, len(mag_path_timestamp_list), len(mag_path_timestamp_list) * 0.0101),
                                                    np.arange(0, len(mag_path_timestamp_list)), mag_path_timestamp_list)
                mag_path_timestamp_df_ = pd.DataFrame(mag_path_timestamp_list_).T
                mag_path_timestamp_df_.columns = [f'mag_z_{str(i)}' for i in range(100)]
                mag_path_timestamp_df_['timestamp'] = timestamp_list[n]
                mag_path_timestamp_df_['path'] = path
                dfs_.append(mag_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/mag_100/z')
    print(building)

5a0546857ecc773753327266
5c3c44b80379370013e0fd2b
5d27075f03f801723c2e360f
5d27096c03f801723c31e5e0
5d27097f03f801723c320d97
5d27099f03f801723c32511d
5d2709a003f801723c3251bf
5d2709b303f801723c327472
5d2709bb03f801723c32852c
5d2709c303f801723c3299ee
5d2709d403f801723c32bd39
5d2709e003f801723c32d896
5da138274db8ce0c98bbd3d2
5da1382d4db8ce0c98bbe92e
5da138314db8ce0c98bbf3a0
5da138364db8ce0c98bc00f1
5da1383b4db8ce0c98bc11ab
5da138754db8ce0c98bca82f
5da138764db8ce0c98bcaa46
5da1389e4db8ce0c98bd0547
5da138b74db8ce0c98bd4774
5da958dd46f8266d0737457b
5dbc1d84c1eb61796cf7c010
5dc8cea7659e181adb076a3f


In [ ]:
#mag_z_test
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/magnetic_field/{building}_magnetic_field_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'mag_z_{str(i)}' for i in range(100)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/mag_100/z')

#rotation_50

In [ ]:
#rotation_x_train
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv')
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    rotation_df = pd.read_csv(f'/content/drive/MyDrive/rotation/{building}_rotation_train.csv')
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        rotation_path_df = rotation_df[rotation_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        if len(timestamp_list) == 0:
            break
        else:
            timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            else:
                timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
                timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            else:
                rotation_path_timestamp_df = rotation_path_df[(rotation_path_df['timestamp'] >= timestamp_05_list[n]) & (rotation_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(acc_path_timestamp_df['x'].values) == 0:
                break
            else:
                rotation_path_timestamp_list = rotation_path_timestamp_df['x'].values
                rotation_path_timestamp_list_ = np.interp(np.arange(0, len(rotation_path_timestamp_list), len(rotation_path_timestamp_list) * 0.0101),
                                                    np.arange(0, len(rotation_path_timestamp_list)), rotation_path_timestamp_list)
                rotation_path_timestamp_df_ = pd.DataFrame(rotation_path_timestamp_list_).T
                rotation_path_timestamp_df_.columns = [f'rotation_x_{str(i)}' for i in range(100)]
                rotation_path_timestamp_df_['timestamp'] = timestamp_list[n]
                rotation_path_timestamp_df_['path'] = path
                dfs_.append(rotation_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/rotation_100/x')
    print(building)

In [ ]:
#rotation_x_test
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/rotation_vector/{building}_rotation_vector_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['x'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['x'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'rotation_x_{str(i)}' for i in range(100)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/rotation_100/x')

In [ ]:
#rotation_y_train
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv')
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    rotation_df = pd.read_csv(f'/content/drive/MyDrive/rotation/{building}_rotation_train.csv')
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        rotation_path_df = rotation_df[rotation_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        if len(timestamp_list) == 0:
            break
        else:
            timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            else:
                timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
                timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            else:
                rotation_path_timestamp_df = rotation_path_df[(rotation_path_df['timestamp'] >= timestamp_05_list[n]) & (rotation_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(acc_path_timestamp_df['y'].values) == 0:
                break
            else:
                rotation_path_timestamp_list = rotation_path_timestamp_df['y'].values
                rotation_path_timestamp_list_ = np.interp(np.arange(0, len(rotation_path_timestamp_list), len(rotation_path_timestamp_list) * 0.0101),
                                                    np.arange(0, len(rotation_path_timestamp_list)), rotation_path_timestamp_list)
                rotation_path_timestamp_df_ = pd.DataFrame(rotation_path_timestamp_list_).T
                rotation_path_timestamp_df_.columns = [f'rotation_y_{str(i)}' for i in range(100)]
                rotation_path_timestamp_df_['timestamp'] = timestamp_list[n]
                rotation_path_timestamp_df_['path'] = path
                dfs_.append(rotation_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/rotation_100/y')
    print(building)

In [ ]:
#rotation_y_test
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/rotation_vector/{building}_rotation_vector_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['y'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['y'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'rotation_y_{str(i)}' for i in range(100)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/rotation_100/y')

In [ ]:
#rotation_z_train
for building in used_buildings:
    train_df = pd.read_csv(f'/content/drive/MyDrive/wifi_train_timestamp_test_path/{building}_train.csv')
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    rotation_df = pd.read_csv(f'/content/drive/MyDrive/rotation/{building}_rotation_train.csv')
    used_path = sorted(train_df['path'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        rotation_path_df = rotation_df[rotation_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        if len(timestamp_list) == 0:
            break
        else:
            timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            else:
                timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
                timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            else:
                rotation_path_timestamp_df = rotation_path_df[(rotation_path_df['timestamp'] >= timestamp_05_list[n]) & (rotation_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            else:
                rotation_path_timestamp_list = rotation_path_timestamp_df['z'].values
                rotation_path_timestamp_list_ = np.interp(np.arange(0, len(rotation_path_timestamp_list), len(rotation_path_timestamp_list) * 0.0101),
                                                    np.arange(0, len(rotation_path_timestamp_list)), rotation_path_timestamp_list)
                rotation_path_timestamp_df_ = pd.DataFrame(rotation_path_timestamp_list_).T
                rotation_path_timestamp_df_.columns = [f'rotation_z_{str(i)}' for i in range(100)]
                rotation_path_timestamp_df_['timestamp'] = timestamp_list[n]
                rotation_path_timestamp_df_['path'] = path
                dfs_.append(rotation_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/rotation_100/z')
    print(building)

In [ ]:
#rotation_z_test
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/rotation_vector/{building}_rotation_vector_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0101),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'rotation_z_{str(i)}' for i in range(100)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/rotation_100/z')